In [ ]:
import astropy.units as u
import astropy.constants as c
from astropy.modeling.physical_models import BlackBody
import numpy as np
import matplotlib.pyplot as plt

# Define SED class
class SED:

    # Initialize the class
    def __init__(self, mag, temperature, temp_units, wave_start, wave_stop, wave_units):
        
        # Define variables
        self.mag = mag
        self.temperature = temperature

        # Define the units of temperature
        if temp_units == 'Kelvin':
            self.temperature = self.temperature*u.K

        # Define the wavelength range
        self.wave_start = wave_start
        self.wave_stop = wave_stop

        # Define the units of wavelength
        if wave_units == "Angstrom":
            self.wave_start = self.wave_start*u.AA
            self.wave_stop = self.wave_stop*u.AA

    def create_spectrum(self):

        '''
        PURPOSE:
                This function creates a blackbody spectrum based on the temperature and magnitude of the star.

        INPUTS:
                        [mag; float]: The magnitude of the star
                [temperature; float]: The temperature of the star
                 [wave_start; float]: The starting wavelength of the spectrum
                  [wave_stop; float]: The ending wavelength of the spectrum

        OUTPUTS:
                          [wavelength; np.array, float]: The wavelength array of the blackbody spectrum
                [normalized_spectrum,; np.array, float]: The normalized blackbody spectrum

        AUTHOR:
                Tim M. Sept. 1 2023.
        '''

        # Create the blackbody spectrum
        blackbody_model = BlackBody(self.temperature, scale = 1*u.erg/u.s/u.cm**2/u.sr/u.AA)

        # Normalize the spectrum
        self.wavelength = np.linspace(self.wave_start, self.wave_stop, 100_000)

        # Define the reference spectrum
        self.model_spectrum = blackbody_model(self.wavelength)
        
        # Normalize the spectrum
        ref_spectrum = BlackBody(temperature = 9_700*u.K, scale = 1*u.erg/u.s/u.cm**2/u.sr/u.AA)

        # Normalize the spectrum
        normalize = (3.63e-9*u.erg/u.cm**2/u.s/u.AA)/(ref_spectrum(5500*u.AA)*u.sr) 

        # Define the normalized spectrum
        self.normalized_spectrum = self.model_spectrum*u.sr*normalize*10.**(-0.4*self.mag)
        
        return self.wavelength, self.normalized_spectrum
    
    def plot(self, plot):

        '''
        PURPOSE:
                This function plots the blackbody spectrum.

        INPUTS:
                [plot; string:  Type of plot

        OUTPUTS:
                Displays a plot of the blackbody spectrum.

        AUTHOR:
                Tim M. Sept. 1 2023.
        '''

        # Create normalized spectrum 
        wavelength, normalized_spectrum = SED.create_spectrum(self)
        
        # Plot the spectrum
        fig,ax = plt.subplots(1, 1, figsize = (10, 8))
        ax.set_xlabel('Wavelength ($\AA$)')
        ax.set_ylabel('Flux Density (ergs $s^{-1} cm^{-2} \AA^{-1}$)')
        ax.plot(np.linspace(self.wave_start, self.wave_stop, 100_000), normalized_spectrum)
        plt.show()

        return 
        
class BandPass:

    # Initialize the bandpass class
    def __init__(self, filter, wavelength, normalized_spectrum):

        # Make a normalized spectrum and wavelength array
        self.normalized_spectrum = normalized_spectrum
        self.wavelength = wavelength

        # For the Johnson V filter
        if filter == 'V':

            # Define the wavelength range
            self.filter_wavelength = np.arange(0, 10_000)

            # Make the filter array of zeros
            self.filter = np.zeros(len(self.filter_wavelength))

            # Make a boxy filter response
            index = np.where((self.filter_wavelength > 5_075) & (self.filter_wavelength < 5_925))
            self.filter[index] = 1
            
    def convolve_SED(self):

        ''' 
        PURPOSE:
                Function that convolves the normalized spectrum with a filter response function.

        INPUTS:
                N/A

        OUTPUTS:
                [convolved_spectrum; np.array, float]:  Normalized spectrum convolved with the filter response.

        AUTHOR:
                Tim M. Sept. 1 2023.
        '''

        # Interpolate the filter response
        interpolated_filter_values = np.interp(self.wavelength.value, self.filter_wavelength, self.filter)
        
        # Multiply the normalized spectrum by the interpolated filter values
        self.convolved_spectrum = self.normalized_spectrum*interpolated_filter_values
        
        return self.convolved_spectrum

In [ ]:
# Initialize the SED class
model = SED(mag = 0, temperature = 9_700, temp_units = 'Kelvin', wave_start = 1, wave_stop = 10_000, wave_units = 'Angstrom')

# Create the spectrum
wavelength,normalized_spectrum = model.create_spectrum()
model.plot('Flux Density')

In [ ]:
# Initialize the BandPass class
um = BandPass('V', wavelength, normalized_spectrum)

In [ ]:
# Convolve the spectrum with the filter response
c = um.convolve_SED()

# Plot the convolved spectrum
plt.plot(wavelength,um.convolved_spectrum)